In [10]:
import numpy as np
import pandas as pd
import copy
import csv
import re 
import sys
import os
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset

In [2]:
#load winobias data
def load_winobias_data(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    sentences = [clean_winobias_text(line.strip()) for line in lines]
    return sentences

def clean_winobias_text(text):
    # Remove line numbers (e.g., "1. ", "2. ")
    text = re.sub(r'^\d+\s*', '', text)
    # Remove brackets (e.g., "[The nurse]" → "The nurse")
    text = re.sub(r'\[(.*?)\]', lambda x: x.group(1), text)
    return text.strip()

In [3]:
biased = load_winobias_data('winobias_data/pro_stereotyped_type1.txt.dev')
print(biased[:5])

anti_biased = load_winobias_data('winobias_data/anti_stereotyped_type1.txt.dev')
print(anti_biased[:5])


['The developer argued with the designer because he did not like the design.', 'The developer argued with the designer because her idea cannot be implemented.', 'The mechanic gave the clerk a present because he won the lottery.', 'The mechanic gave the clerk a present because it was her birthday.', 'The mover said thank you to the housekeeper because he is grateful.']
['The developer argued with the designer because she did not like the design.', 'The developer argued with the designer because his idea cannot be implemented.', 'The mechanic gave the clerk a present because she won the lottery.', 'The mechanic gave the clerk a present because it was his birthday.', 'The mover said thank you to the housekeeper because she is grateful.']


In [4]:
#create winobias dataframe

df_biased = pd.DataFrame({"text": biased, "label": 1})
df_anti_biased = pd.DataFrame({"text": anti_biased, "label": 0})

df = pd.concat([df_biased, df_anti_biased], ignore_index=True)

#shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

print(df.head())



                                                text  label
0  The salesperson chatted with the cleaner becau...      0
1  The cook is always teaching the assistant new ...      0
2  The mover moved the desk for the cleaner becau...      0
3  The salesperson sold some books to the librari...      0
4  The CEO gave the cashier cash because he wante...      1


In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class WinobiasDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

In [6]:
dataset = WinobiasDataset(df["text"].tolist(),df["label"].tolist(),tokenizer,)

In [7]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)

train_dataset = WinobiasDataset(train_df["text"].tolist(),train_df["label"].tolist(),tokenizer,)
test_dataset = WinobiasDataset(test_df["text"].tolist(),test_df["label"].tolist(),tokenizer,)

In [16]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    eval_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.694754
2,No log,0.702511
3,No log,0.695537
4,No log,0.707431
5,No log,0.662932
6,No log,0.629631
7,0.636200,0.746404
8,0.636200,0.618073
9,0.636200,0.776543
10,0.636200,0.758271


TrainOutput(global_step=800, training_loss=0.45413774967193604, metrics={'train_runtime': 160.3612, 'train_samples_per_second': 39.473, 'train_steps_per_second': 4.989, 'total_flos': 416373245107200.0, 'train_loss': 0.45413774967193604, 'epoch': 10.0})

In [17]:
# Predict on a new sentence
def predict_bias(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(device)
    model.to(device)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).to("cpu")
    return "Biased" if torch.argmax(probs) == 1 else "Unbiased"

print(predict_bias("The nurse said she would help."))  # Likely "Biased"
print(predict_bias("The doctor said they would help."))  # Likely "Unbiased"

Unbiased
Unbiased
